In [65]:
import pandas as pd

data1 = pd.read_csv('../../raw/통합 테이블(공원)_2019.csv')
data2 = pd.read_csv('../../raw/지역사회건강조사 (전처리된 데이터)_2019/통합 테이블(건강)_2019.csv')
data2.rename(columns={'지역':'자치구'},inplace = True)
df2 = data2[data2['연령별'] == '45~64']
df2 = df2[df2['성별'] == '계']
df2.set_index('자치구',inplace = True)
data1.set_index('자치구',inplace = True)
df = pd.concat([df2,data1],axis=1)
df = df.drop(df.index[0])
df = df.drop(['연령별','성별'],axis=1)

In [66]:
import requests
import json
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins

In [67]:
seoul_region = '../../raw/서울시 행정구역 코드.csv'
region_data = pd.read_csv(seoul_region, encoding = 'euc-kr')
#seoul_data = state_data[['NAME','POPULATION']]
region_data.set_index('자치구',inplace = True)
total_data = pd.concat([region_data,df],axis=1)
total_data.reset_index(inplace=True)
total_data.head()

,자치구,Code,주관적 건강수준,걷기 실천,격렬한 신체활동 실천,긍정적 태도율(자연환경),주관적 스트레스 수준,우울감 경험률,중등도 신체활동 실천,체중조절 경험 여부,...,행정구역면적,공원면적,공원율,1인당 공원 면적,총 도시림 면적,생활권 도시림 면적,1인당 총 도시림 면적,1인당 생활권 도시림 면적,1인당 생활권 도시림 면적(산림수목),1인당 생활권 도시림 면적(공원녹지)
0,종로구,11110,33.8,41.0,14.2,83.1,24.5,1.8,8.5,67.3,...,"23,913,280","11,556,112",48.33,71.4,13277114.30,2501869.30,87.759365,16.536911,0.639714,15.897197
1,중구,11140,35.0,69.9,26.9,82.3,19.9,2.4,12.3,64.2,...,"9,959,983","3,155,139",31.68,23.1,5417625.70,1506737.70,42.938755,11.942029,1.278750,10.663278
2,용산구,11170,50.0,75.0,24.1,85.3,12.1,1.2,13.8,76.2,...,"21,866,384","1,779,562",8.14,7.3,2005971.00,809094.00,8.772340,3.538260,2.208309,1.329951
3,성동구,11200,28.4,61.9,20.1,66.6,28.4,6.1,17.1,71.3,...,"16,859,343","3,100,047",18.39,10.0,1388926.00,1184497.00,4.616074,3.936658,1.076404,2.860254
4,광진구,11215,36.2,54.3,18.8,78.0,19.0,6.8,14.5,67.1,...,"17,062,949","3,363,481",19.71,9.2,6073160.28,1287528.28,17.285215,3.664518,1.901005,1.763512


In [68]:
state_geo = '../../raw/seoul-maps-master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [69]:
json_data=open(state_geo,encoding='utf-8-sig').read()
jsonResult = json.loads(json_data)

# 중앙위치 계산
def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

# 다중 Array 구조 이중으로 변환
def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

center_locations = pd.DataFrame()
codes = []
regions = []
x_list = []
y_list = []

for x in range(0, len(jsonResult['features'])):
    code = jsonResult['features'][x]['properties']['code']
    name = jsonResult['features'][x]['properties']['name']
    # 중앙값 생성
    points = jsonResult['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    # 결과
    codes.append(code)
    regions.append(name)
    x_list.append(X)
    y_list.append(Y)

# 데이터 프레임 생성
center_locations['Code'] = codes
center_locations['자치구'] = regions
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [70]:
center_locations2 = center_locations[center_locations['Y'] >= 37.426026]
target_df = pd.merge(total_data,center_locations2, how = 'left', on = '자치구')
target_df = target_df.dropna(axis=0, subset=['X','Y']) # 위치 정보 없는 값 제외

In [71]:
state_geo = json.load(open('../../raw/seoul-maps-master/kostat/2013/json/seoul_municipalities_geo_simple.json',encoding='utf-8-sig'))
# Initialize the map:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=state_geo,
    name='1인당 생활권 도시림 면적',
    data=total_data,
    columns=['자치구', '1인당 생활권 도시림 면적'],
    key_on='feature.properties.name',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.3,
    color = 'gray',
    legend_name = 'income'
)

# I can add marker one by one on the map
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)

    if target_df.iloc[i]['자치구'] in ['서초구','강남구'] :
        color = 'white'
    else:
        color = '#3186cc'

    folium.CircleMarker(location, radius=10,color=color,fill_color=color,fill_opacity = 0.5, opacity=0.0,  popup=target_df.iloc[i]['자치구'] + "\n" + str((round(target_df.iloc[i]['1인당 생활권 도시림 면적'],2))) + "㎡/인").add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

# Save to html
m.save('kr_incode.html')
# webbrowser.open_new("folium_kr.html")
m

C:\Users\gimhu\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
